# This notebook is to read the CLAGNs from the literature with their redshifts and types
Last edit: Oct 4th, 2023

In [1]:
import pandas as pd
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy import table
from astropy.table import Table, join, join_skycoord
from astroquery.ipac.ned import Ned
from astroquery.vizier import Vizier
from astroquery.simbad import Simbad
import sys
sys.path.append('../code/')
from sample_selection import clean_sample
from sample_lc import parallel_lc

In [2]:
# LaMassa 2015, Macleod 2016
macleod_CSQ = Table.read('https://academic.oup.com/mnras/article/457/1/389/989199', htmldict={'table_id': 5}, format='ascii.html')
coord_str = macleod_CSQ['Name\n            .'][0]
test_str = coord_str[0:2]+ " "+ coord_str[2:4]+ " " + coord_str[4:9] + " " + coord_str[9:12] + " " + coord_str[12:14]+ " " + coord_str[14:]
c = SkyCoord(test_str, unit=(u.hourangle, u.deg))
d={'ra':[c.ra.degree],'dec':[c.dec.degree],'redshift':[macleod_CSQ['z\n            .'][0]],'turnon':[1]}

k=1
ton,toff = 1,0
for i in range(1,len(macleod_CSQ)):
    coord_str = macleod_CSQ['Name\n            .'][i]
    test_str = coord_str[0:2]+ " "+ coord_str[2:4]+ " " + coord_str[4:9] + " " + coord_str[9:12] + " " + coord_str[12:14]+ " " + coord_str[14:]
    c = SkyCoord(test_str, unit=(u.hourangle, u.deg))
    d['ra'].append(c.ra.degree)
    d['dec'].append(c.dec.degree)
    d['redshift'].append(macleod_CSQ['z\n            .'][i])
    label = macleod_CSQ['BEL\n            .'][i]
    if label=='Appear':
        d['turnon'].append(1)
        ton+=1
    else:
        d['turnon'].append(0)  
        toff+=1
    k+=1
print(str(k)+' added, turn_on=',str(ton)+', turn_off='+str(toff))        

10 added, turn_on= 4, turn_off=6


In [3]:
# Ruan 2016
ruan_CSQ = Ned.query_refcode('2016ApJ...826..188R')
ruan_coords = [SkyCoord(ra, dec, frame='icrs', unit='deg') for ra, dec in zip(ruan_CSQ['RA'], ruan_CSQ['DEC'])]
k=0
ton,toff = 0,0

for i in range(len(ruan_CSQ['RA'])):
    coord = SkyCoord(ruan_coords[i], unit=(u.hourangle, u.deg))
    d['ra'].append(coord.ra.degree)
    d['dec'].append(coord.dec.degree)
    d['redshift'].append(ruan_CSQ['Redshift'][i])
    d['turnon'].append(0)
    k+=1
    toff+=1

print(str(k)+' added, turn_on=',str(ton)+', turn_off='+str(toff))        

3 added, turn_on= 0, turn_off=3


In [4]:
#Yang 2018
CLQ = np.genfromtxt('yangg.cat', dtype=[('SDSSid', 'S18'), ('RA', 'S18'), ('DEC', 'S18'),
                                        ('Redshift','<f8'),('Transition','S18')])
k=0
ton,toff = 0,0
for i in range(len(CLQ['RA'])):
    ra = CLQ['RA'][i].decode('utf8')
    dec = CLQ['DEC'][i].decode('utf8')    
    c = SkyCoord(ra+dec, unit=(u.hourangle, u.deg))
    d['ra'].append(c.ra.degree)
    d['dec'].append(c.dec.degree)
    d['redshift'].append(CLQ['Redshift'][i])
    if CLQ['Transition'][i]==b'Turn-on':
        d['turnon'].append(1)
        ton+=1
    else:
        d['turnon'].append(0)
        toff+=1

    k+=1
print(str(k)+' added, turn_on=',str(ton)+', turn_off='+str(toff))        

26 added, turn_on= 16, turn_off=10


In [5]:
# MacLeod2019
a = np.genfromtxt('table2.dat', dtype=[('SDSSid', 'S18'), ('redshift', '<f8'), ('morphology', '<i8'),
                                      ('mjd1', 'S24'), ('mag1', '<f8'),('dmag1', '<f8'),
                                      ('mjd2', 'S24'), ('mag2', '<f8'),('dmag2', '<f8'),
                                      ('mjds', 'S24'), ('mjdps1', 'S24'), ('magps', '<f8'),('dmagps', '<f8'),
                                       ('mjdps2', 'S24'), ('tel', 'S24'), ('clq', '<i8'),('nsigma', '<f8')])
k=0
ton,toff = 0,0

for i in range(len(a['SDSSid'])):
    coord_str = a['SDSSid'][i].decode('utf8')
    test_str = coord_str[0:2]+ " "+ coord_str[2:4]+ " " + coord_str[4:9] + " " + coord_str[9:12] + " " + coord_str[12:14]+ " " + coord_str[14:]
    coords = SkyCoord(test_str, unit=(u.hourangle, u.deg))
    
    d['ra'].append(coords.ra.degree)
    d['dec'].append(coords.dec.degree)
    d['redshift'].append(a['redshift'][i])

    if a['mag1'][i]>a['mag2'][i]:
        d['turnon'].append(1)
        ton+=1

    else:
        d['turnon'].append(0)
        toff+=1

    k+=1
print(str(k)+' added, turn_on=',str(ton)+', turn_off='+str(toff))        

262 added, turn_on= 17, turn_off=245


In [6]:
#Sheng 
CLQ = Ned.query_refcode('2020ApJ...889...46S') 
sheng_CLQ = CLQ[[0,1,2,3,5,6]]#need the first 3 objects in their table, 
sheng_coords = [SkyCoord(ra, dec, frame='icrs', unit='deg') for ra, dec in zip(sheng_CLQ['RA'], sheng_CLQ['DEC'])]
k=0
ton,toff = 0,0
for i in range(len(sheng_CLQ['Redshift'])):
    d['ra'].append(sheng_coords[i].ra.degree)
    d['dec'].append(sheng_coords[i].dec.degree)
    d['redshift'].append(sheng_CLQ['Redshift'][i])
    d['turnon'].append(0)
    k+=1
    toff+=1
    
print(str(k)+' added, turn_on=',str(ton)+', turn_off='+str(toff))        

6 added, turn_on= 0, turn_off=6


In [7]:
#green
Vizier.ROW_LIMIT = -1
catalog_list = Vizier.find_catalogs('J/ApJ/933/180')
catalogs = Vizier.get_catalogs(catalog_list.keys())
table2 = catalogs[0]
green_CSQ = table2.to_pandas()
k=0
ton,toff = 0,0

for i in range(len(green_CSQ)):
    if "CLQ" in (green_CSQ['Notes'][i]):

        d['ra'].append(green_CSQ['_RA'][i])
        d['dec'].append(green_CSQ['_DE'][i])
        d['redshift'].append(green_CSQ['zspec'][i])
        if "TurnOn" in (green_CSQ['Notes'][i]):
            d['turnon'].append(1)
            ton+=1
        else:
            d['turnon'].append(0)
            toff+=1
        k+=1
print(str(k)+' added, turn_on=',str(ton)+', turn_off='+str(toff))        

19 added, turn_on= 4, turn_off=15


In [8]:
#Lyu
CLQ = Ned.query_refcode('2022ApJ...927..227L') 
#CLQ['RA'], CLQ['DEC'], CLQ['Redshift'], CLQ['Type']

In [9]:
result_table = Simbad.query_bibobj('2022MNRAS.513L..57L')
result_table = result_table[[0,1,2,3]]  #pick the correct sources by hand
result_table.columns

k=0
ton,toff = 0,0
z = [0.1022,0.2375,0.1083,0.0849]
for i in range(len(result_table)):
    d['ra'].append(result_table['RA'][i])
    d['dec'].append(result_table['DEC'][i])
    d['redshift'].append(z[i])
    d['turnon'].append(1)
    ton+=1
    k+=1
print(str(k)+' added, turn_on=',str(ton)+', turn_off='+str(toff))  

4 added, turn_on= 4, turn_off=0


In [10]:
CLQ = Ned.query_refcode('2022MNRAS.511...54H') 
#CLQ.columns
ids = [2,3,5,6,10,11,12,13,16,20,24,25,27,28,14,26]
onoff = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0]
k=0
ton,toff = 0,0
for i,idd in enumerate(ids):    
    d['ra'].append(CLQ['RA'][idd])
    d['dec'].append(CLQ['DEC'][idd])
    d['redshift'].append(CLQ['Redshift'][idd])
    if onoff[i]==1:
        d['turnon'].append(1)
        ton+=1
    else:
        d['turnon'].append(0)
        toff+=1
    k+=1
print(str(k)+' added, turn_on=',str(ton)+', turn_off='+str(toff))  

16 added, turn_on= 14, turn_off=2


In [11]:
df = pd.DataFrame(data=d)
df

,ra,dec,redshift,turnon
0,5.796083,0.588194,0.422000,1
1,29.990167,0.552889,0.312000,0
2,36.483625,0.507417,0.504000,0
3,36.717667,-0.654583,0.625000,0
4,150.584042,45.157583,0.400000,0
...,...,...,...,...
341,309.4995,-50.39264,0.063087,1
342,347.33004,-32.49933,0.054167,1
343,352.63454,-2.46251,0.033430,1
344,205.064,-4.89214,0.086537,0


In [12]:
d1 = df[df['turnon']==1]
d0 = df[df['turnon']==0]
print(len(d1),len(d0))

coords = [SkyCoord(ra, dec, frame='icrs', unit='deg') for ra, dec in zip(df['ra'], df['dec'])]
labels=[]
for l in df['turnon']:
    if l==1:
        labels.append('CL-On')
    else:
        labels.append('CL-Off')

c,l = clean_sample(coords,labels)

59 287
after duplicates removal, sample size: 325


In [13]:
dflc = parallel_lc(c,l,parquet_savename = '../output/df_lc_changelook.parquet')


  2%|▏         | 6/325 [00:05<05:54,  1.11s/it]No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (5.79608333, 0.58819444)>".
  2%|▏         | 7/325 [00:06<06:56,  1.31s/it]No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (29.99016667, 0.55288889)>".
  2%|▏         | 8/325 [00:09<08:42,  1.65s/it]No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (36.483625, 0.50741667)>".
  3%|▎         | 9/325 [00:10<07:56,  1.51s/it]No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (36.71766667, -0.65458333)>".
  4%|▍         | 13/325 [00:12<03:56,  1.32it/s]No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (150.58404167, 45.15758333)>".
  0%|          | 0/442 [00:00<?, ?it/s]1.55it/s]No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (155.46808333, 46.75433333)>".
  6%|▌         | 19/325 [00:15<02:45,  1.85it/s]No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (201.23870833, 48.04477778)>".
  6%|▋

Loading:  IC40_exp.csv
Loading:  IC59_exp.csv
Loading:  IC79_exp.csv
Loading:  IC86_III_exp.csv
Loading:  IC86_II_exp.csv
Loading:  IC86_IV_exp.csv
Loading:  IC86_I_exp.csv
Loading:  IC86_VII_exp.csv
Loading:  IC86_VI_exp.csv
Loading:  IC86_V_exp.csv
done


  0%|          | 6/1344 [00:04<14:24,  1.55it/s]No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (354.01241667, 0.29130556)>".
  6%|▋         | 21/325 [00:02<00:34,  8.84it/s]No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (127.884375, 36.77144444)>".
 14%|█▍        | 45/325 [00:04<00:29,  9.47it/s]No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (132.49075, 27.79136111)>".
  1%|          | 13/1344 [00:10<22:11,  1.00s/it]No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (137.38341667, 47.79183333)>".
 27%|██▋       | 88/325 [00:09<00:24,  9.75it/s]]No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (144.37633333, 26.04225)>".
 30%|███       | 99/325 [00:10<00:23,  9.52it/s]]No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (150.84779167, 35.41772222)>".
 39%|███▉      | 127/325 [00:13<00:20,  9.86it/s]No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (166.229875, 1.31572222)>".


working on mission FERMIGTRIG
working on mission SAXGRBMGRB
INFO: Query finished. [astroquery.utils.tap.core]

Search completed in 2.78 seconds
Number of objects matched: 293 out of 325.
<TableColumns names=('ra','dec','random_index','source_id','ra_2','dec_2','objectid')>
IceCube Matched and added to lightcurve object.
0 no matches
1 no matches
2 no matches
3 no matches
4 no matches
5 no matches
6 no matches
7 no matches
8 no matches
9 no matches
10 no matches
11 no matches
12 no matches
13 no matches
14 no matches
15 no matches
16 no matches
17 no matches
18 no matches
19 no matches
20 no matches
21 no matches
22 no matches
23 no matches
24 no matches
25 no matches
26 no matches
27 no matches
28 no matches
29 no matches
30 no matches
31 no matches
32 no matches
33 no matches
34 no matches
35 no matches
36 no matches
37 no matches
38 no matches
39 no matches
40 no matches
41 no matches
42 no matches
43 no matches
44 no matches
45 no matches
46 no matches
47 no matches
48 no matches
49

## 